# Домашнее задание №4

# Выполнил Салабай В.Р. (ФПэ-01-22)

По аналогии с решением задач из практики, построить график зависимости $\eta_{oi} = f(H_0)$ в диапазоне $H_0$ = (50 - 150) $\frac {kJ}{kg}$. $\eta_{ол}$ = 78%. $u = 160 m/s$. Все остальные переменные и условия принять такие же как на практическом заняти.

In [100]:
#Загружаем библиотеки
import iapws
from iapws import IAPWS97 as gas
import math
import matplotlib.pyplot as plt
import numpy as np

In [101]:
p = 5 * MPa
t = to_kelvin(489)
H_0 = range(50,150)
u = 160
d_r = 1.09
s_div_r = 0.2
degree__of_reaction = 0.1
z_bandage = 2
delta_r_bandage = 1.17 / 1000
delta_a_bandage = 4 / 1000
z_rotor = 5
d_leak_rotor = 0.36
delta_leak_rotor = 0.4 / 1000
e = 0.8
sinus_alpha1 = 0.225
segments = 4
F1 = 0.025
K_y = 1
mu_r_rotor = 0.8
mu_r = mu_r_rotor
mu_a = 0.5
mu_soplo = 0.97
blade_width = 0.035
blade_length = 0.035
blade_kpd = 0.78

In [102]:
#СИ
MPa = 10 ** 6
kPa = 10 ** 3
unit = 1 / MPa
to_kelvin = lambda x: x + 273.15 if x else None

In [103]:
point = gas(P=p * unit, T=t)
kinematic_vyazkost = point.nu

In [104]:
def Re_number1(u, d_r):
        return  u * d_r * 0.5 / kinematic_vyazkost

In [105]:
def k_frictions(s_div_r, re):
        return 2.5 * 10 ** (-2) * s_div_r ** 0.1 * re **(-0.2)

In [106]:
def friction_loss(s_div_r, d_r, u, kinematic_vyazkost, u_div_dummy_speed, F1):
    Re_number = Re_number1(u,d_r)
    k = k_frictions(s_div_r, Re_number)
    friction_loss_pu = k * d_r ** 2 * u_div_dummy_speed ** 3 / F1 
    return friction_loss_pu

In [107]:
def ventilation_loss(m, k, sinus, e, u_div_dummy_speed):
    first = k / sinus
    second = (1 - e) / e
    third = u_div_dummy_speed ** 3
    return first * second * third * m

In [108]:
def get_segment_loss_pu(B, l, F, u_div_dummy_speed, blade_kpd, segments):
    first = 0.25 * B * l / F
    second = u_div_dummy_speed * blade_kpd * segments
    return first * second

In [109]:
def get_partial_losses_pu(u_div_dummy_speed, blade_width, blade_length, F1, blade_kpd, segments):
    ventilation_loss_pu = ventilation_loss(
    m = 1,
    k = 0.065,
    e = e,
    u_div_dummy_speed = u_div_dummy_speed,
    sinus = sinus_alpha1
    )
    segment_loss_pu = get_segment_loss_pu(
    B = blade_width,
    l = blade_length,
    F = F1,
    u_div_dummy_speed = u_div_dummy_speed,
    blade_kpd = blade_kpd,
    segments = segments
    )
    partial_losses_pu = segment_loss_pu + ventilation_loss_pu
    return partial_losses_pu

In [110]:
def compute_equal_gap(z, delta_r, mu_r_rotor, delta_a, mu_a):
    first = 1 / (mu_a * delta_a) ** 2
    second = z / (mu_r_rotor * delta_r) ** 2
    return (first + second) ** (-0.5)

In [111]:
def bandage_leak_loss(d_shroud, delta_eq, F, dor, l, kpd):
    d_avg = d_shroud - l
    first = math.pi * d_shroud * delta_eq / F
    second = dor + 1.8 * (l / d_avg)
    return first * (second) ** 0.5 * kpd

In [112]:
def disk_leak_loss(K, F, mu_r_rotor, mu_soplo, F_soplo, z, kpd):
    upper = mu_r_rotor * K * F * kpd
    lower = mu_soplo * F_soplo * z ** 0.5
    return upper / lower

In [113]:
def leak_losses(z_bandage, delta_r_bandage, mu_r_rotor, delta_a_bandage, z_rotor, mu_a, mu_soplo, F1, degree__of_reaction, blade_length, delta_leak_rotor, d_leak_rotor, blade_kpd, K_y):
    d_shroud = delta_r_bandage / 0.001 
    delta_eq_bandage = compute_equal_gap(
        z = z_bandage,
        delta_r = delta_r_bandage,
        mu_r_rotor = mu_r_rotor,
        delta_a = delta_a_bandage,
        mu_a = mu_a
    )
    bandage_leak_loss_pu = bandage_leak_loss(
        d_shroud = d_shroud,
        delta_eq = delta_eq_bandage,
        F = F1,
        dor = degree__of_reaction,
        l = blade_length,
        kpd = blade_kpd
        )
    F_leak_rotor = math.pi * d_leak_rotor * delta_leak_rotor
    disk_leak_loss_pu = disk_leak_loss(
        K = K_y,
        F = F_leak_rotor,
        mu_r_rotor = mu_r_rotor,
        mu_soplo = mu_soplo,
        F_soplo = F1,
        z = z_rotor,
        kpd = blade_kpd
    )
    leak_losses_pu = disk_leak_loss_pu + bandage_leak_loss_pu
    return leak_losses_pu

In [114]:
def internal_kpd(blade_kpd, friction_loss_pu, partial_losses_pu, leak_losses_pu):
    internal_kpd = blade_kpd - friction_loss_pu - partial_losses_pu - leak_losses_pu
    return internal_kpd

In [115]:
leak_losses_pu = leak_losses(
    z_bandage = z_bandage,
    delta_r_bandage = delta_r_bandage,
    mu_r_rotor = mu_r_rotor,
    delta_a_bandage = delta_a_bandage,
    z_rotor = z_rotor,
    mu_a = mu_a,
    mu_soplo = mu_soplo,
    F1 = F1,
    degree__of_reaction = degree__of_reaction,
    blade_length = blade_length,
    delta_leak_rotor = delta_leak_rotor,
    d_leak_rotor  = d_leak_rotor,
    blade_kpd = blade_kpd,
    K_y = K_y
)

In [116]:
internal_kpd = []
for H_0znachenie in H_0:
    H_0znachenie=H_0znachenie*1000
    dummy_speed = (2 * H_0znachenie) ** 0.5
    u_div_dummy_speed = u / dummy_speed
    friction_loss_pu = friction_loss(
        s_div_r = s_div_r,
        d_r = d_r, 
        u = u,
        kinematic_vyazkost = kinematic_vyazkost,
        u_div_dummy_speed = u_div_dummy_speed,
        F1 = F1
    )
    partial_losses_pu = get_partial_losses_pu(
        u_div_dummy_speed = u_div_dummy_speed,
        blade_width = blade_width,
        blade_length = blade_length,
        F1 = F1,
        blade_kpd = blade_kpd,
        segments = segments
    )
    internal_kpd.append(internal_kpd(blade_kpd,friction_loss_pu,partial_losses_pu,leak_losses_pu))

In [117]:
#Построение графика
plt.plot(H_0,internal_kpd)  
plt.xlabel("$H_{0}$")
plt.ylabel('$\eta_{oi}$')
plt.title("Зависимость $\eta_{oi}$ от значения $H_{0}$")
plt.grid()